In [ ]:
with open("../datasets/jupyterhub/hub.log","r") as f:
    lines = f.readlines()

linecount = len(lines)
errorcount = 0
for line in lines:
    if line.startswith("[E"):
        #print(line.strip())
        errorcount +=1
        
print(linecount,errorcount,errorcount/linecount)

In [ ]:
from datetime import datetime 
from typing import List
def extractTimeStamp(tokens :List[str]):
    '''
    Assuming line has valid timestamp in the first place...
    line = "[I 2021-11-20 18:05:06.587 JupyterHub log:189] "
    '''
    try:
        extractedDateTimeString  = tokens[1] + " " + tokens[2]
    except IndexError:
        return None
    
    try:
        return  datetime.fromisoformat(extractedDateTimeString)
    except ValueError:
        try:
            return datetime.strptime(extractedDateTimeString, "%Y%m%d %H:%M:%S.%f")
        except ValueError:
            try:
                return datetime.strptime("20" + extractedDateTimeString,"%Y%m%d %H:%M:%S")
            except ValueError:
                return None


assert extractTimeStamp("[I 2021-11-20 18:05:06.587 JupyterHub log:189] ".split())==datetime(2021, 11, 20, 18, 5, 6, 587000)
assert extractTimeStamp("[I 20211120 18:05:06.587 JupyterHub log:189] ".split())==datetime(2021, 11,20,18,5,6,587000)
assert extractTimeStamp("[I 20211120 18:05:06 JupyterHub log:189] ".split())==None
assert extractTimeStamp(" [I 211121 00:42:43 __init__:190] Culling server".split())==datetime(2021,11,21,0,42,43,0)

In [ ]:
extractTimeStamp("[I 2021-11-20 18:07:08.587 JupyterHub base:909] User dlnosky took 5.795 seconds to start")

In [ ]:
def extractLevel(tokens : List[str]):
    if len(tokens) == 0: return None
    if tokens[0].startswith("[I"):
        level = "INFO"
    elif tokens[0].startswith("[E"):
        level = "ERROR"
    elif tokens[0].startswith("[W"):
        level = "WARN"
    else:
        level = None
    return level 

assert extractLevel("[I 2021-11-20 18:05:06.587 JupyterHub log:189] ".split()) == "INFO"
assert extractLevel("[E 2021-11-20 18:05:06.587 JupyterHub log:189] ".split()) == "ERROR"
assert extractLevel("[W 2021-11-20 18:05:06.587 JupyterHub log:189] ".split()) == "WARN"
assert extractLevel("2021-11-20 18:05:06.587 JupyterHub log:189] ".split()) == None

In [ ]:
import re
import string 

def extractNetid(line :str):
    '''
    Several cases to yank out a netid from a line
    
    generic searches happen last
    '''
    searches = [
        { "pattern": r"User \w+ took \d+.\d+ seconds to start", "split": " ", "index": 1 },
        { "pattern": r"User \w+ server took \d+.\d+ seconds to stop", "split": " ", "index": 1 },   
        { "pattern": r"User \w+\: server is slow to stop", "split": " ", "index": 1 },           
        { "pattern" : r"\w+ still running$", "split": " ","index": 0 },
        { "pattern": r"Removing user \w+ from proxy", "split": " ", "index": 2 },
        { "pattern": r"/hub/api/users/\w+", "split": "/", "index": -1 },
        { "pattern": r"Adding user \w+ to proxy", "split": " ", "index": 2 },
        { "pattern": r"Server \w+ is ready", "split": " ", "index": 1 },
        { "pattern": r"\w+ is pending spawn", "split": " ", "index": 0 },
        { "pattern": r"User logged \w+: \w+", "split": " ", "index": -1 },
        { "pattern": r"Culling server \w+", "split": " ", "index": -1 },
        { "pattern": r"Failed to cleanup \w+", "split": " ", "index": -1 },
        { "pattern": r"\w+ does not appear to be running", "split": " ", "index": 0 },

        { "pattern": r"claim-\w+", "split": "-", "index": -1 },
        { "pattern": r"jupyter\-\w+", "split": "-", "index": -1 },
        { "pattern": r"jupyterhub-user-\w+", "split": "-", "index": -1 },
        { "pattern": r"/user/\w+", "split": "/", "index": -1 },
        { "pattern": r"%2Fuser%2F\w+", "split": "%2F", "index": -1 },
        { "pattern": r"\w+@\d+.\d+.\d+.\d+", "split": "@", "index": 0 }        
    ]
    netid = None
    for search in searches:
        match = re.search(search["pattern"],line)
        if match:
            extract = line[match.start():match.end()].split(search["split"])[search["index"]]
            netid = re.sub(r'[^\w\s]', '', extract)
            break
    
    return netid
    
                                                                
assert extractNetid("[I 2021-11-20 17:37:33.081 JupyterHub app:2186] yhuan161 still running")=='yhuan161'
assert extractNetid("[I 2021-11-20 17:37:45.659 JupyterHub proxy:309] Removing user amccaffe from proxy (/user/amccaffe/)")=="amccaffe"
assert extractNetid("[I 2021-11-20 17:38:13.279 JupyterHub spawner:2620] Deleting pod jupyterhub/jupyter-cma11")=="cma11"
assert extractNetid("[I 2021-11-20 17:38:24.712 JupyterHub log:189] 200 POST /hub/api/users/mafudge/activity (mafudge@10.42.4.154) 37.28ms") == "mafudge"
assert extractNetid("[I 2021-11-20 17:38:41.143 JupyterHub log:189] 200 GET /hub/error/503?url=%2Fuser%2Fcma110%2Fapi%2Fkernels%2Fd29763ae-64e9-4e58-ad2f-3b547108ec73%2Fchannels%3Fsession_id%3De4af78f1784e49db807c3bbb91dc00b5 (@10.42.11.53) 6.13ms") == "cma110"
assert extractNetid("[I 2021-11-20 17:39:22.077 JupyterHub log:189] 302 GET /hub/ -> /hub/spawn (enbilyns@45.47.85.5) 24.18ms")=='enbilyns'
assert extractNetid("[I 2021-11-20 17:39:26.137 JupyterHub spawner:2361] PVC claim-enbilyns already exists, so did not create new pvc")=='enbilyns'
assert extractNetid("[I 2021-11-20 17:39:26.137 JupyterHub spawner:2361] PVC claim-enbilyns already exists, so did not create new pvc")=='enbilyns'
assert extractNetid("[W 2021-11-20 22:17:53.102 JupyterHub base:1148] User dlnosky: server is slow to stop (timeout=10)")=='dlnosky'
assert extractNetid("[I 2021-11-20 22:17:53.102 JupyterHub log:189] 202 DELETE /hub/api/users/dlnosky/server (cull-idle@127.0.0.1) 10015.55ms")=='dlnosky'
assert extractNetid("[I 2021-11-20 22:17:58.573 JupyterHub base:1115] User dlnosky server took 15.472 seconds to stop")=='dlnosky'



In [ ]:
# Sessions
user = "dlnosky"
events_to_track =  { "LOGIN": 'User logged in:' , "START" :  "seconds to start", "STOP":"seconds to stop", "CULL":"Culling server" } #, "ACTIVITY":f"/hub/api/users/{user}/activity"}
sessionStartTimestamp = None 
for line in lines:
    if line.startswith("[I") and line.find(user) >= 0:
        tokens = line.strip().split()
        event = None
        elapsedTime = None
        if line.find(events_to_track["LOGIN"])>=0:
            event="LOGIN"
            loginTimestamp = extractTimeStamp(tokens)
        elif line.find(events_to_track["START"])>=0:
            event="START"
            sessionStartTimestamp = extractTimeStamp(tokens)
        elif line.find(events_to_track["STOP"])>=0:
            event="STOP"
            sessionStopTimestamp = extractTimeStamp(tokens)
            elapsedTime = sessionStopTimestamp - sessionStartTimestamp if sessionStartTimestamp is not None else None
        elif line.find(events_to_track["CULL"])>=0:
            event="CULL"
            
        if event is not None:
            print(f"[** {event:8} **] {line.strip()}")
        if elapsedTime is not None:
            print(f"SESSION LENGTH: {elapsedTime}")
                    

In [65]:
# Extract Event
def extractEvent(line:str,netid:str):
    events_to_track =  { 
        "ACTIVITY":f"/hub/api/users/{netid}/activity",
        "LOGIN": "User logged in:", 
        "START" : "seconds to start", 
        "STOP": "seconds to stop", 
        "CULL":"Culling server"
    }
    for key in events_to_track.keys():
        if line.find(events_to_track[key])>=0:
            return key
    return None

In [71]:
session = {}
logs = []
for line in lines:
    elapsedTime = None
    tokens = line.strip().split()
    level = extractLevel(tokens)
    timestamp = extractTimeStamp(tokens)
    netid = extractNetid(line)
    event = extractEvent(line,netid)
    if event == "START":
        session[netid] = timestamp 
    elif event == "STOP" and session.get(netid,None) is not None:        
        elapsedTime = timestamp - session[netid]
        del session[netid]

    # require at LEAST a level and timestamp before writing a log
    if level is not None and timestamp is not None:
        log = { "level": level, 
               "timestamp" : timestamp.isoformat(), 
               "netid" : netid, 
               "event" : event, 
               "sessionminutes" : elapsedTime.seconds/60 if elapsedTime is not None else None, 
               "data" : line.strip() 
              }
        logs.append(log)

In [76]:
import pandas as pd
df = pd.DataFrame(logs)
df.to_json("/home/jovyan/datasets/jupyterhub/cleaned-logs.json", orient="records")

In [ ]:
TODO: Exercise with line by line feeding to topic.
INFO|WARN|ERROR datetimestamp username data

- KSQL make a stream
- KSQL Code to find the START sessions
- KSQL Code to find the CULL sessions
- KSQL to count the errors each hour 

In [ ]:
user = "dlnosky"
skiplines = ['[W ','[E ','/hub/oauth_login',f'/hub/api/users/{user}/activity','hub/api/authorizations/token/']

events_to_track = [ { "LOGIN", 'User logged in:' } , {"START","seconds to start"}, {"STOP","seconds to stop"}, {"CULL","Culling server"},{"ACTIVITY",f"/hub/api/users/{user}/activity"}]

for line in lines:
    skip = False
    if line.find(user) >= 0:
        if line.find("User logged in:") >=0:
            char = "[LOGIN] =======> "
        elif line.find("seconds to start") >=0:
            char = "[START] =======> "
        elif line.find("seconds to stop") >=0:
            char = "[STOP ] =======> "
        elif line.find("Culling server") >=0:
            char = "[CULL ] =======> "
        else:
            char = ""
            
        for skipline in skiplines:
            if line.find(skipline) >=0:
                skip = True

        if not skip:
            print(char+ line.strip())
        

```mermaid

    Logs (cleaned logs) --> Kafka 
    Kafka --> (Errors) --> Elastic
    Kafka --> (All Events) --> Cassandra
    Kafka --> (Session Lengths) --> Elastic? 

```